In [1]:
import os

import numpy as np
from tensorflow.keras.models import *

from src.dlla.berg import make_mlp
from src.dlla.hw import prepare_traces_dl, dlla_known_p, shuffle
from src.dlla.wegener import binomial_test, make_mlp_wegener
from src.trace_set.database import Database
from src.trace_set.pollution import Pollution, PollutionType
from src.trace_set.set_hw import TraceSetHW

In [2]:
from src.tvla.cri import rho_test, tvla_t_test


def store_results(database: Database, method: str, pollution: Pollution, p):
    file_name = f"results_{database.name}.csv"
    with open(file_name, 'a') as f:
        f.write(f"{method};{pollution.type.name};{pollution.parameter};{p}\n")

In [3]:
LIMITS = {
    Database.ascad: (None, 1000),
    Database.ascad_none: (None, 1000),
    Database.aisy: (1000, 1000),
}

PARAMS = {
    Database.ascad: {
        PollutionType.desync: np.arange(0, 2.05, .05),
        PollutionType.gauss: np.arange(0, 5.1, .1)
    },
    Database.ascad_none: {
        PollutionType.desync: np.arange(0, 205, 5),
        PollutionType.gauss: np.arange(0, 102, 2)
    },
    Database.aisy: {
        PollutionType.desync: np.arange(0, 460, 10),
        PollutionType.gauss: np.union1d(np.arange(0, 4100, 100), np.arange(0, 410, 10))
    }
}

def get_params(db: Database, pollution_type: PollutionType):
    return PARAMS[db][pollution_type]

def get_limit(db: Database):
    return LIMITS[db]

def get_order(db: Database):
    if db is Database.ascad:
        return 2

    return 3

def wegener_p(mdl: Model, x_att: np.ndarray, y_att: np.ndarray):
    predictions = mdl.predict(x_att).argmax(axis=1)
    labels = y_att.argmax(axis=1)

    correct = np.sum(predictions == labels)
    total = len(predictions)
    # print(total, correct)

    return binomial_test(total, correct)

def la_benchmark(db: Database, pollution_type: PollutionType, params):
    for param in params:
        pollution = Pollution(pollution_type, param)
        print("Load traces         ", end="\r")
        prof_lim, att_lim = get_limit(db)
        trace_set = TraceSetHW(db, pollution, (prof_lim, att_lim))

        if os.path.exists(trace_set.path):
            print("Prepare traces (1/3)", end="\r")
            x9, y9, x9_att, y9_att = prepare_traces_dl(*trace_set.profile(), *trace_set.attack())
            print("Prepare traces (3/3)", end="\r")
            x2b, y2b, x2b_att, y2b_att = prepare_traces_dl(*trace_set.profile_la(), *trace_set.attack_la(True), num_classes=2)

            print("Make model (1/3)    ", end="\r")
            mdl_open = make_mlp(x9, y9, progress=False)
            print("Make model (3/3)    ", end="\r")
            mdl_wegener = make_mlp_wegener(x2b, y2b, False)

            tvla_x, tvla_y = shuffle(*trace_set.profile_la())
            print("TVLA                ", end="\r")
            tvla_ps = np.min(tvla_t_test(tvla_x[:att_lim], tvla_y[:att_lim], get_order(db), progress=False), axis=1)
            rho_p = np.min(rho_test(tvla_x[:att_lim], tvla_y[:att_lim], progress=False), axis=1)[0]

            print("Predict             ", end="\r")
            dlla_k_p = dlla_known_p(mdl_open, x9_att, y9_att)
            dlla_wegener_p = wegener_p(mdl_wegener, x2b_att, y2b_att)

            title = f"{pollution_type} ({param})."
            print(f"{title} TVLA t ({tvla_ps}). TVLA rho ({rho_p}). DLLA known key ({dlla_k_p}). DLLA Wegener ({dlla_wegener_p})")

            for order, p in enumerate(tvla_ps):
                if order > 0:
                    store_results(db, f"tvla_t_{order}", pollution, p)

            store_results(db, "tvla_rho", pollution, rho_p)
            store_results(db, "dlla_open", pollution, dlla_k_p)
            store_results(db, "dlla_wegener", pollution, dlla_wegener_p)

        print()

la_benchmark(Database.ascad_none, PollutionType.desync, [0])

In [4]:
def la_benchmark_all(db: Database):
    for poll in [PollutionType.desync, PollutionType.gauss]:
        la_benchmark(db, poll, get_params(db, poll))

while True:
    la_benchmark_all(Database.aisy)
    la_benchmark_all(Database.ascad)
    la_benchmark_all(Database.ascad_none)

PollutionType.desync (0.0). DL-LA unknown key (4.6499758199686715e-102)
PollutionType.desync (0.05). DL-LA unknown key (0.9493880689792986)
PollutionType.desync (0.1). DL-LA unknown key (1.342737898536477e-75)
PollutionType.desync (0.15000000000000002). DL-LA unknown key (1.1447864118008835e-105)
PollutionType.desync (0.2). DL-LA unknown key (1.1891438365552593e-157)
PollutionType.desync (0.25). DL-LA unknown key (4.421697409745384e-245)
PollutionType.desync (0.30000000000000004). DL-LA unknown key (7.974987500539961e-06)
PollutionType.desync (0.35000000000000003). DL-LA unknown key (1.103059583944319e-43)
PollutionType.desync (0.4). DL-LA unknown key (1.6497286362237143e-133)
PollutionType.desync (0.45). DL-LA unknown key (0.22170204929345683)
PollutionType.desync (0.5). DL-LA unknown key (0.01210842002179598)
PollutionType.desync (0.55). DL-LA unknown key (3.042263898446831e-23)
PollutionType.desync (0.6000000000000001). DL-LA unknown key (1.3406091246657658e-110)
PollutionType.desyn

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/berg/anaconda3/envs/LA/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-8e6e1776c4e1>", line 7, in <module>
    la_benchmark_all(Database.ascad)
  File "<ipython-input-4-8e6e1776c4e1>", line 3, in la_benchmark_all
    la_benchmark(db, poll, get_params(db, poll))
  File "<ipython-input-3-a18e09957332>", line 63, in la_benchmark
    mdl_closed = make_mlp_berg(x2, y2, progress=False, num_classes=2)
  File "/home/berg/Documents/Afstudeer/LA/src/dlla/berg.py", line 34, in make_mlp_berg
    return build_mlp(x, y, {
  File "/home/berg/Documents/Afstudeer/LA/src/dlla/berg.py", line 23, in build_mlp
    out = mdl.fit(x, y, shuffle=True, batch_size=params['batch_size'], epochs=params['epochs'],
  File "/home/berg/anaconda3/envs/LA/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py", line 1183, in fit
    tmp_logs 

TypeError: object of type 'NoneType' has no len()